In [6]:
import numpy as np
import pandas as pd
#import requests
# from bs4 import BeautifulSoup
# from collections import Counter
import re
import itertools
import string
import sys
# %matplotlib inline
# import matplotlib.pyplot as plt
# import seaborn as sns
# sns.set_style('whitegrid')

# from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
# from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
# from nltk.stem.snowball import SnowballStemmer
# from nltk.stem.wordnet import WordNetLemmatizer
# from nltk.corpus import wordnet
# from ast import literal_eval
# from fractions import Fraction
# #from surprise import Reader, Dataset, SVD, evaluate

In [7]:


def filter_data(df, position, min_snap_pct, min_games):


    ## Filtering the dataframe.. just want to work with QBs for now 
    qb_df = df.loc[df['starter_pos'] == position].reset_index(drop=True)
    
    
    ## converting game location to home and away 
    qb_df['game_location'] = qb_df['game_location'].apply(lambda x: 'home' if pd.isnull(x) else 'away')
    
    ### Dropping any rows with NaN at offensive snap percentage 
    qb_df.drop(qb_df[qb_df['off_pct'].isnull()].index, inplace=True)

    ## convert strings to percentages 
    qb_df['off_pct'] = qb_df['off_pct'].apply(lambda x: float(x.strip('%')))
    qb_df['def_pct'] = qb_df['def_pct'].apply(lambda x: float(x.strip('%')))
    qb_df['st_pct'] = qb_df['st_pct'].apply(lambda x: float(x.strip('%')))

    ### converting game result column to four columns.. result,total_scorem, points_for, points_against
    ### 


    qb_df['result'] = qb_df['game_result'].apply(lambda x: re.split(' |-', x)[0])
    qb_df['pf'] = qb_df['game_result'].apply(lambda x: int(re.split(' |-', x)[1]))
    qb_df['pa'] = qb_df['game_result'].apply(lambda x: int(re.split(' |-', x)[2]))
    qb_df['ptotal'] = qb_df['pf'] + qb_df['pa']
    ### Now let's select playes who have played more than x number of games and y average snap count percentage in a year 
    test = qb_df.loc[qb_df['off_pct'] > min_snap_pct].reset_index(drop=True)


    #### There are these outliers like sam bradford who are morons and dunno what to do with them
    ### Maybe later i can remove any game with less than 5.0 fantasy points and more than 30 or something fantasy 
    ### points NEED TO THINK MORE ABOUT THIS LATERSS 
    ## test.loc[test['draftkings_points'] == 2.6000]


    game_count = test['name'].value_counts()
    game_count = game_count.loc[game_count.values > min_games]
    sel_players = game_count.keys().tolist()

    fin_qb_df = test.loc[test['name'].isin(sel_players)].reset_index(drop=True)
    
    #### Team names need to be consistent as a couple of teams changed their names recently 
    fin_qb_df['team'] = fin_qb_df['team'].apply(lambda x: 'LAR' if x == 'STL' else x)
    fin_qb_df['opp'] = fin_qb_df['opp'].apply(lambda x: 'LAR' if x == 'STL' else x)
    fin_qb_df['team'] = fin_qb_df['team'].apply(lambda x: 'LAC' if x == 'SDG' else x)
    fin_qb_df['opp'] = fin_qb_df['opp'].apply(lambda x: 'LAC' if x == 'SDG' else x)
    
    
    
    ########### DROPPING GAME 16 BECAUSE THAT IS WEEK 17 AND IS USELESS 
    ### Dropping any rows with NaN at offensive snap percentage 
    fin_qb_df.drop(fin_qb_df[fin_qb_df['game_num'] == 16].index, inplace=True)
    
    

    ## we only need some of the columns selecting them first 
    fin_qb_df = fin_qb_df.drop(['special_teams', 'fantasy_points', 'fanduel_points', 'game_result','defense','def_pct',
               'st_pct','lnk_id','pf','pa','ptotal','result','off_pct','offense'], axis=1)


    

    return fin_qb_df 

In [8]:
#### Compiling Training data
### We are going to use 2011 to 2017 data to train the model and then test on 2018 data 

yr_list = np.arange(2012,2019,1)

##yr_list = [2017]
position = 'QB'
min_snap_pct = 75.0
#min_snap_pct = 90.0
min_games = 8
#min_games = 10
tr_df = pd.DataFrame(columns=())
for yr in yr_list:
    print(yr)
    fname = 'total_profootball_'+str(yr)+'_fantasy_data.csv'
    tdf = pd.read_csv(fname,index_col=[0])
    tdf = filter_data(tdf,position,min_snap_pct,min_games)
    tr_df = pd.concat([tr_df, tdf], axis=0).reset_index(drop=True)

    

    

# ### For now removing anything that is more than two standard deviations less than the mean draftkings points
tr_df = tr_df.loc[(tr_df['draftkings_points'] >= 2.44)]

2012
2013
2014
2015
2016
2017
2018


In [10]:

tr_df.describe()

,draftkings_points,game_num,year
count,2667.000000,2667.000000,2667.000000
mean,18.769104,7.987627,2015.018748
std,8.041458,4.288131,2.009455
min,2.600000,1.000000,2012.000000
25%,12.950000,4.000000,2013.000000
50%,17.900000,8.000000,2015.000000
75%,23.900000,12.000000,2017.000000
max,49.300000,15.000000,2018.000000


In [11]:
## creating labels 

hpt = int(tr_df['draftkings_points'].quantile(0.75).round(0))
mpt = int(tr_df['draftkings_points'].quantile(0.50).round(0))
lpt = int(tr_df['draftkings_points'].quantile(0.25).round(0))


tr_df['dfk_labels'] = [0]*len(tr_df)

tr_df['dfk_labels'] = tr_df['draftkings_points'].apply(lambda x: 1 if x >= hpt else(2 if x <= hpt and x>=mpt else (4 if x <= lpt else 3)))
 

In [126]:
# ### GAME COUNT 
# tr_df['pl_gc'] = [1]*len(tr_df)

# def player_game_count(tr_df):
#     new_df = pd.DataFrame(columns=())
#     for i in tr_df.name.unique():
#         tmpdf = tr_df.loc[tr_df['name'] == i]
#         ##tmpdf = tmpdf.sort_values(['game_date'], ascending=[1]).reset_index(drop=True)
#         t1 = tmpdf.sort_values(['game_date'],ascending =[1]).reset_index(drop=True)
#         for i in range(len(t1)):
#             if i == 0:
#                 t1.loc[i,['pl_gc']] = 1
#             else:
#                 t1.loc[i,['pl_gc']] = t1.loc[i,['pl_gc']] + t1.loc[i-1,['pl_gc']] 
#                 ## this means that the player is a rookie  
#         new_df = pd.concat([new_df,t1],axis=0).reset_index(drop=True)
    
#     return new_df
    

In [127]:
# tr_df = player_game_count(tr_df)

In [128]:

# ## we only need some of the columns selecting them first 
# tr_df = tr_df.drop(['special_teams', 'fantasy_points', 'fanduel_points', 'game_result','defense','def_pct',
#            'st_pct','lnk_id'], axis=1)



In [129]:
# tr_df.loc[tr_df['name'] == 'Drew Brees']

,game_date,opp,starter_pos,draftkings_points,team,game_num,game_location,name,year,dfk_labels,pl_gc
0,2012-09-09,WAS,QB,26.6,NOR,1,home,Drew Brees,2012.0,1,1
1,2012-09-16,CAR,QB,24.0,NOR,2,away,Drew Brees,2012.0,1,2
2,2012-09-23,KAN,QB,20.5,NOR,3,home,Drew Brees,2012.0,2,3
3,2012-09-30,GNB,QB,32.8,NOR,4,away,Drew Brees,2012.0,1,4
4,2012-10-07,LAC,QB,32.6,NOR,5,home,Drew Brees,2012.0,1,5
5,2012-10-21,TAM,QB,33.2,NOR,6,away,Drew Brees,2012.0,1,6
6,2012-10-28,DEN,QB,15.7,NOR,7,away,Drew Brees,2012.0,3,7
7,2012-11-05,PHI,QB,16.5,NOR,8,home,Drew Brees,2012.0,3,8
8,2012-11-11,ATL,QB,22.8,NOR,9,home,Drew Brees,2012.0,2,9
9,2012-11-18,OAK,QB,20.8,NOR,10,away,Drew Brees,2012.0,2,10


In [130]:
# ### Dividing into training and testing dataframes 

# test_df = tr_df.loc[tr_df['year'] == 2018].reset_index(drop=True)
# tr_df.drop(tr_df[tr_df['year'] == 2018].index, inplace =True)      

In [131]:
tr_df

,game_date,opp,starter_pos,draftkings_points,team,game_num,game_location,name,year,dfk_labels,pl_gc
0,2012-09-09,WAS,QB,26.6,NOR,1,home,Drew Brees,2012.0,1,1
1,2012-09-16,CAR,QB,24.0,NOR,2,away,Drew Brees,2012.0,1,2
2,2012-09-23,KAN,QB,20.5,NOR,3,home,Drew Brees,2012.0,2,3
3,2012-09-30,GNB,QB,32.8,NOR,4,away,Drew Brees,2012.0,1,4
4,2012-10-07,LAC,QB,32.6,NOR,5,home,Drew Brees,2012.0,1,5
5,2012-10-21,TAM,QB,33.2,NOR,6,away,Drew Brees,2012.0,1,6
6,2012-10-28,DEN,QB,15.7,NOR,7,away,Drew Brees,2012.0,3,7
7,2012-11-05,PHI,QB,16.5,NOR,8,home,Drew Brees,2012.0,3,8
8,2012-11-11,ATL,QB,22.8,NOR,9,home,Drew Brees,2012.0,2,9
9,2012-11-18,OAK,QB,20.8,NOR,10,away,Drew Brees,2012.0,2,10


In [95]:
# tr_df['lb1'] = [0]*len(tr_df)
# tr_df['lb2'] = [0]*len(tr_df)
# tr_df['lb3'] = [0]*len(tr_df)
# tr_df['lb4'] = [0]*len(tr_df)

# tr_df['lb1'] = tr_df['dfk_labels'].apply(lambda x: 1 if x == 1 else 0)
# tr_df['lb2'] = tr_df['dfk_labels'].apply(lambda x: 1 if x == 2 else 0)
# tr_df['lb3'] = tr_df['dfk_labels'].apply(lambda x: 1 if x == 3 else 0)
# tr_df['lb4'] = tr_df['dfk_labels'].apply(lambda x: 1 if x == 4 else 0)


# tr_df['ngames'] = [1]*len(tr_df)

In [96]:
# tr_df['lb1_p'] = [0]*len(tr_df)
# tr_df['lb2_p'] = [0]*len(tr_df)
# tr_df['lb3_p'] = [0]*len(tr_df)
# tr_df['lb4_p'] = [0]*len(tr_df)
# tr_df['ngames_p'] = [1]*len(tr_df)

In [97]:
tr_df

,off_pct,game_date,opp,starter_pos,offense,draftkings_points,team,game_num,game_location,name,...,lb1,lb2,lb3,lb4,ngames,lb1_p,lb2_p,lb3_p,lb4_p,ngames_p
0,100.0,2012-09-09,WAS,QB,72.0,26.6,NOR,1,home,Drew Brees,...,1,0,0,0,1,0,0,0,0,1
1,100.0,2012-09-16,CAR,QB,78.0,24.0,NOR,2,away,Drew Brees,...,1,0,0,0,1,0,0,0,0,1
2,100.0,2012-09-23,KAN,QB,60.0,20.5,NOR,3,home,Drew Brees,...,0,1,0,0,1,0,0,0,0,1
3,100.0,2012-09-30,GNB,QB,78.0,32.8,NOR,4,away,Drew Brees,...,1,0,0,0,1,0,0,0,0,1
4,100.0,2012-10-07,LAC,QB,71.0,32.6,NOR,5,home,Drew Brees,...,1,0,0,0,1,0,0,0,0,1
5,100.0,2012-10-21,TAM,QB,64.0,33.2,NOR,6,away,Drew Brees,...,1,0,0,0,1,0,0,0,0,1
6,100.0,2012-10-28,DEN,QB,61.0,15.7,NOR,7,away,Drew Brees,...,0,0,1,0,1,0,0,0,0,1
7,100.0,2012-11-05,PHI,QB,56.0,16.5,NOR,8,home,Drew Brees,...,0,0,1,0,1,0,0,0,0,1
8,100.0,2012-11-11,ATL,QB,62.0,22.8,NOR,9,home,Drew Brees,...,0,1,0,0,1,0,0,0,0,1
9,86.0,2012-11-18,OAK,QB,49.0,20.8,NOR,10,away,Drew Brees,...,0,1,0,0,1,0,0,0,0,1


In [98]:
### prob features


# def prob_features(tr_df):
#     new_df = pd.DataFrame(columns=())
#     for i in tr_df.name.unique():
#         tmpdf = tr_df.loc[tr_df['name'] == i]
#         ##tmpdf = tmpdf.sort_values(['game_date'], ascending=[1]).reset_index(drop=True)
#         t1 = tmpdf.sort_values(['game_date'],ascending =[1]).reset_index(drop=True)
#         for i in range(len(t1)):
#             if i == 0:
#                 t1.loc[i,['lb1','lb2','lb3','lb4','ngames']] = t1.loc[i,['lb1','lb2','lb3','lb4','ngames']] + [1,1,1,1,4]
#             else:
#                 t1.loc[i,['lb1','lb2','lb3','lb4','ngames']] = t1.loc[i,['lb1','lb2','lb3','lb4','ngames']] + t1.loc[i-1,['lb1','lb2','lb3','lb4','ngames']] 
#                 ## this means that the player is a rookie  
#         new_df = pd.concat([new_df,t1],axis=0).reset_index(drop=True)
    
#     return new_df

In [99]:

# def prob_features(tr_df):
#     new_df = pd.DataFrame(columns=())
#     for i in tr_df.name.unique():
#         tmpdf = tr_df.loc[tr_df['name'] == i]
#         ##tmpdf = tmpdf.sort_values(['game_date'], ascending=[1]).reset_index(drop=True)
#         t1 = tmpdf.sort_values(['game_date'],ascending =[1]).reset_index(drop=True)
#         for i in range(len(t1)):
#             if i == 0:
#                 t1.loc[i,['lb1_p','lb2_p','lb3_p','lb4_p','ngames_p']] = [1,1,1,1,4]
#             else:
#                 t1.loc[i,'lb1_p'] = t1.loc[i-1,'lb1'] + t1.loc[i-1,'lb1_p'] 
#                 t1.loc[i,'lb2_p'] = t1.loc[i-1,'lb2'] + t1.loc[i-1,'lb2_p']
#                 t1.loc[i,'lb3_p'] = t1.loc[i-1,'lb3'] + t1.loc[i-1,'lb3_p']
#                 t1.loc[i,'lb4_p'] = t1.loc[i-1,'lb4'] + t1.loc[i-1,'lb4_p']
#                 t1.loc[i,'ngames_p'] = t1.loc[i-1,'ngames'] + t1.loc[i-1,'ngames_p']
#                 ## this means that the player is a rookie  
#         new_df = pd.concat([new_df,t1],axis=0).reset_index(drop=True)
    
#     return new_df

In [100]:
# tr_df = prob_features(tr_df)

In [101]:
tr_df

,off_pct,game_date,opp,starter_pos,offense,draftkings_points,team,game_num,game_location,name,...,lb1,lb2,lb3,lb4,ngames,lb1_p,lb2_p,lb3_p,lb4_p,ngames_p
0,100.0,2012-09-09,WAS,QB,72.0,26.6,NOR,1,home,Drew Brees,...,1,0,0,0,1,1,1,1,1,4
1,100.0,2012-09-16,CAR,QB,78.0,24.0,NOR,2,away,Drew Brees,...,1,0,0,0,1,2,1,1,1,5
2,100.0,2012-09-23,KAN,QB,60.0,20.5,NOR,3,home,Drew Brees,...,0,1,0,0,1,3,1,1,1,6
3,100.0,2012-09-30,GNB,QB,78.0,32.8,NOR,4,away,Drew Brees,...,1,0,0,0,1,3,2,1,1,7
4,100.0,2012-10-07,LAC,QB,71.0,32.6,NOR,5,home,Drew Brees,...,1,0,0,0,1,4,2,1,1,8
5,100.0,2012-10-21,TAM,QB,64.0,33.2,NOR,6,away,Drew Brees,...,1,0,0,0,1,5,2,1,1,9
6,100.0,2012-10-28,DEN,QB,61.0,15.7,NOR,7,away,Drew Brees,...,0,0,1,0,1,6,2,1,1,10
7,100.0,2012-11-05,PHI,QB,56.0,16.5,NOR,8,home,Drew Brees,...,0,0,1,0,1,6,2,2,1,11
8,100.0,2012-11-11,ATL,QB,62.0,22.8,NOR,9,home,Drew Brees,...,0,1,0,0,1,6,2,3,1,12
9,86.0,2012-11-18,OAK,QB,49.0,20.8,NOR,10,away,Drew Brees,...,0,1,0,0,1,6,3,3,1,13


In [102]:
# tr_df['lb1'] = tr_df['lb1']/tr_df['ngames']
# tr_df['lb2'] = tr_df['lb2']/tr_df['ngames']
# tr_df['lb3'] = tr_df['lb3']/tr_df['ngames']
# tr_df['lb4'] = tr_df['lb4']/tr_df['ngames']

In [103]:
tr_df

,off_pct,game_date,opp,starter_pos,offense,draftkings_points,team,game_num,game_location,name,...,lb1,lb2,lb3,lb4,ngames,lb1_p,lb2_p,lb3_p,lb4_p,ngames_p
0,100.0,2012-09-09,WAS,QB,72.0,26.6,NOR,1,home,Drew Brees,...,1,0,0,0,1,1,1,1,1,4
1,100.0,2012-09-16,CAR,QB,78.0,24.0,NOR,2,away,Drew Brees,...,1,0,0,0,1,2,1,1,1,5
2,100.0,2012-09-23,KAN,QB,60.0,20.5,NOR,3,home,Drew Brees,...,0,1,0,0,1,3,1,1,1,6
3,100.0,2012-09-30,GNB,QB,78.0,32.8,NOR,4,away,Drew Brees,...,1,0,0,0,1,3,2,1,1,7
4,100.0,2012-10-07,LAC,QB,71.0,32.6,NOR,5,home,Drew Brees,...,1,0,0,0,1,4,2,1,1,8
5,100.0,2012-10-21,TAM,QB,64.0,33.2,NOR,6,away,Drew Brees,...,1,0,0,0,1,5,2,1,1,9
6,100.0,2012-10-28,DEN,QB,61.0,15.7,NOR,7,away,Drew Brees,...,0,0,1,0,1,6,2,1,1,10
7,100.0,2012-11-05,PHI,QB,56.0,16.5,NOR,8,home,Drew Brees,...,0,0,1,0,1,6,2,2,1,11
8,100.0,2012-11-11,ATL,QB,62.0,22.8,NOR,9,home,Drew Brees,...,0,1,0,0,1,6,2,3,1,12
9,86.0,2012-11-18,OAK,QB,49.0,20.8,NOR,10,away,Drew Brees,...,0,1,0,0,1,6,3,3,1,13


In [104]:
# tr_df['lb1_p'] = tr_df['lb1_p']/tr_df['ngames_p']
# tr_df['lb2_p'] = tr_df['lb2_p']/tr_df['ngames_p']
# tr_df['lb3_p'] = tr_df['lb3_p']/tr_df['ngames_p']
# tr_df['lb4_p'] = tr_df['lb4_p']/tr_df['ngames_p']

In [105]:
tr_df

,off_pct,game_date,opp,starter_pos,offense,draftkings_points,team,game_num,game_location,name,...,lb1,lb2,lb3,lb4,ngames,lb1_p,lb2_p,lb3_p,lb4_p,ngames_p
0,100.0,2012-09-09,WAS,QB,72.0,26.6,NOR,1,home,Drew Brees,...,1,0,0,0,1,0.250000,0.250000,0.250000,0.250000,4
1,100.0,2012-09-16,CAR,QB,78.0,24.0,NOR,2,away,Drew Brees,...,1,0,0,0,1,0.400000,0.200000,0.200000,0.200000,5
2,100.0,2012-09-23,KAN,QB,60.0,20.5,NOR,3,home,Drew Brees,...,0,1,0,0,1,0.500000,0.166667,0.166667,0.166667,6
3,100.0,2012-09-30,GNB,QB,78.0,32.8,NOR,4,away,Drew Brees,...,1,0,0,0,1,0.428571,0.285714,0.142857,0.142857,7
4,100.0,2012-10-07,LAC,QB,71.0,32.6,NOR,5,home,Drew Brees,...,1,0,0,0,1,0.500000,0.250000,0.125000,0.125000,8
5,100.0,2012-10-21,TAM,QB,64.0,33.2,NOR,6,away,Drew Brees,...,1,0,0,0,1,0.555556,0.222222,0.111111,0.111111,9
6,100.0,2012-10-28,DEN,QB,61.0,15.7,NOR,7,away,Drew Brees,...,0,0,1,0,1,0.600000,0.200000,0.100000,0.100000,10
7,100.0,2012-11-05,PHI,QB,56.0,16.5,NOR,8,home,Drew Brees,...,0,0,1,0,1,0.545455,0.181818,0.181818,0.090909,11
8,100.0,2012-11-11,ATL,QB,62.0,22.8,NOR,9,home,Drew Brees,...,0,1,0,0,1,0.500000,0.166667,0.250000,0.083333,12
9,86.0,2012-11-18,OAK,QB,49.0,20.8,NOR,10,away,Drew Brees,...,0,1,0,0,1,0.461538,0.230769,0.230769,0.076923,13


In [106]:
# ##### Defense susceptibility matrix 
# tr_df['lb1_d'] = [0]*len(tr_df)
# tr_df['lb2_d'] = [0]*len(tr_df)
# tr_df['lb3_d'] = [0]*len(tr_df)
# tr_df['lb4_d'] = [0]*len(tr_df)
# tr_df['ngames_d'] = [1]*len(tr_df)




# def prob_features_def(tr_df):
#     new_df = pd.DataFrame(columns=())
#     for i in tr_df.opp.unique():
#         tmpdf = tr_df.loc[tr_df['opp'] == i]
#         ##tmpdf = tmpdf.sort_values(['game_date'], ascending=[1]).reset_index(drop=True)
#         t1 = tmpdf.sort_values(['game_date'],ascending =[1]).reset_index(drop=True)
#         for i in range(len(t1)):
#             if i == 0:
#                 t1.loc[i,['lb1_d','lb2_d','lb3_d','lb4_d','ngames_d']] = [1,1,1,1,4]
#             else:
#                 t1.loc[i,'lb1_d'] = t1.loc[i-1,'lb1'] + t1.loc[i-1,'lb1_d'] 
#                 t1.loc[i,'lb2_d'] = t1.loc[i-1,'lb2'] + t1.loc[i-1,'lb2_d']
#                 t1.loc[i,'lb3_d'] = t1.loc[i-1,'lb3'] + t1.loc[i-1,'lb3_d']
#                 t1.loc[i,'lb4_d'] = t1.loc[i-1,'lb4'] + t1.loc[i-1,'lb4_d']
#                 t1.loc[i,'ngames_d'] = t1.loc[i-1,'ngames'] + t1.loc[i-1,'ngames_d']
#                 ## this means that the player is a rookie  
#         new_df = pd.concat([new_df,t1],axis=0).reset_index(drop=True)
    
#     return new_df




In [107]:
# tr_df = prob_features_def(tr_df)

In [108]:
# tr_df['lb1_d'] = tr_df['lb1_d']/tr_df['ngames_d']
# tr_df['lb2_d'] = tr_df['lb2_d']/tr_df['ngames_d']
# tr_df['lb3_d'] = tr_df['lb3_d']/tr_df['ngames_d']
# tr_df['lb4_d'] = tr_df['lb4_d']/tr_df['ngames_d']



In [109]:
# tr_df['lb1_h'] = [0]*len(tr_df)
# tr_df['lb2_h'] = [0]*len(tr_df)
# tr_df['lb3_h'] = [0]*len(tr_df)
# tr_df['lb4_h'] = [0]*len(tr_df)
# tr_df['ngames_h'] = [1]*len(tr_df)

# tr_df['lb1_a'] = [0]*len(tr_df)
# tr_df['lb2_a'] = [0]*len(tr_df)
# tr_df['lb3_a'] = [0]*len(tr_df)
# tr_df['lb4_a'] = [0]*len(tr_df)
# tr_df['ngames_a'] = [1]*len(tr_df)






# def prob_features_home_away(tr_df):
#     new_df = pd.DataFrame(columns=())
#     for i in tr_df.name.unique():
#         tmpdf = tr_df.loc[(tr_df['name'] == i)]
#         ##tmpdf = tmpdf.sort_values(['game_date'], ascending=[1]).reset_index(drop=True)
#         t1 = tmpdf.sort_values(['game_date'],ascending =[1]).reset_index(drop=True)
#         for i in range(len(t1)):
#             if i == 0:
#                 t1.loc[i,['lb1_a','lb2_a','lb3_a','lb4_a','ngames_a']] = [1,1,1,1,4]
#                 t1.loc[i,['lb1_h','lb2_h','lb3_h','lb4_h','ngames_h']] = [1,1,1,1,4]
#             else:
#                 if t1.loc[i,'game_location'] == 'home':
#                     t1.loc[i,'lb1_h'] = t1.loc[i-1,'lb1'] + t1.loc[i-1,'lb1_h'] 
#                     t1.loc[i,'lb2_h'] = t1.loc[i-1,'lb2'] + t1.loc[i-1,'lb2_h']
#                     t1.loc[i,'lb3_h'] = t1.loc[i-1,'lb3'] + t1.loc[i-1,'lb3_h']
#                     t1.loc[i,'lb4_h'] = t1.loc[i-1,'lb4'] + t1.loc[i-1,'lb4_h']
#                     t1.loc[i,'ngames_h'] = t1.loc[i-1,'ngames'] + t1.loc[i-1,'ngames_h']
#                     t1.loc[i,'lb1_a'] =  t1.loc[i-1,'lb1_a'] 
#                     t1.loc[i,'lb2_a'] =  t1.loc[i-1,'lb2_a']
#                     t1.loc[i,'lb3_a'] =  t1.loc[i-1,'lb3_a']
#                     t1.loc[i,'lb4_a'] =  t1.loc[i-1,'lb4_a']
#                     t1.loc[i,'ngames_a'] = t1.loc[i-1,'ngames_a']
                    
#                 else:
#                     t1.loc[i,'lb1_a'] = t1.loc[i-1,'lb1'] + t1.loc[i-1,'lb1_a'] 
#                     t1.loc[i,'lb2_a'] = t1.loc[i-1,'lb2'] + t1.loc[i-1,'lb2_a']
#                     t1.loc[i,'lb3_a'] = t1.loc[i-1,'lb3'] + t1.loc[i-1,'lb3_a']
#                     t1.loc[i,'lb4_a'] = t1.loc[i-1,'lb4'] + t1.loc[i-1,'lb4_a']
#                     t1.loc[i,'ngames_a'] = t1.loc[i-1,'ngames'] + t1.loc[i-1,'ngames_a']
#                     t1.loc[i,'lb1_h'] =  t1.loc[i-1,'lb1_h'] 
#                     t1.loc[i,'lb2_h'] =  t1.loc[i-1,'lb2_h']
#                     t1.loc[i,'lb3_h'] =  t1.loc[i-1,'lb3_h']
#                     t1.loc[i,'lb4_h'] =  t1.loc[i-1,'lb4_h']
#                     t1.loc[i,'ngames_h'] = t1.loc[i-1,'ngames_h']
                    
#                 ## this means that the player is a rookie  
#         new_df = pd.concat([new_df,t1],axis=0).reset_index(drop=True)
    
#     return new_df


In [110]:
# tr_df = prob_features_home_away(tr_df)

In [111]:
# tr_df

,off_pct,game_date,opp,starter_pos,offense,draftkings_points,team,game_num,game_location,name,...,lb1_h,lb2_h,lb3_h,lb4_h,ngames_h,lb1_a,lb2_a,lb3_a,lb4_a,ngames_a
0,100.0,2012-09-09,WAS,QB,72.0,26.6,NOR,1,home,Drew Brees,...,1,1,1,1,4,1,1,1,1,4
1,100.0,2012-09-16,CAR,QB,78.0,24.0,NOR,2,away,Drew Brees,...,1,1,1,1,4,2,1,1,1,5
2,100.0,2012-09-23,KAN,QB,60.0,20.5,NOR,3,home,Drew Brees,...,2,1,1,1,5,2,1,1,1,5
3,100.0,2012-09-30,GNB,QB,78.0,32.8,NOR,4,away,Drew Brees,...,2,1,1,1,5,2,2,1,1,6
4,100.0,2012-10-07,LAC,QB,71.0,32.6,NOR,5,home,Drew Brees,...,3,1,1,1,6,2,2,1,1,6
5,100.0,2012-10-21,TAM,QB,64.0,33.2,NOR,6,away,Drew Brees,...,3,1,1,1,6,3,2,1,1,7
6,100.0,2012-10-28,DEN,QB,61.0,15.7,NOR,7,away,Drew Brees,...,3,1,1,1,6,4,2,1,1,8
7,100.0,2012-11-05,PHI,QB,56.0,16.5,NOR,8,home,Drew Brees,...,3,1,2,1,7,4,2,1,1,8
8,100.0,2012-11-11,ATL,QB,62.0,22.8,NOR,9,home,Drew Brees,...,3,1,3,1,8,4,2,1,1,8
9,86.0,2012-11-18,OAK,QB,49.0,20.8,NOR,10,away,Drew Brees,...,3,1,3,1,8,4,3,1,1,9


In [112]:
# tr_df['lb1_h'] = tr_df['lb1_h']/tr_df['ngames_h']
# tr_df['lb2_h'] = tr_df['lb2_h']/tr_df['ngames_h']
# tr_df['lb3_h'] = tr_df['lb3_h']/tr_df['ngames_h']
# tr_df['lb4_h'] = tr_df['lb4_h']/tr_df['ngames_h']


# tr_df['lb1_a'] = tr_df['lb1_a']/tr_df['ngames_a']
# tr_df['lb2_a'] = tr_df['lb2_a']/tr_df['ngames_a']
# tr_df['lb3_a'] = tr_df['lb3_a']/tr_df['ngames_a']
# tr_df['lb4_a'] = tr_df['lb4_a']/tr_df['ngames_a']

In [404]:
#### dropping rookies from test_Df 

# test_df['lb1'] = [0]*len(test_df)
# test_df['lb2'] = [0]*len(test_df)
# test_df['lb3'] = [0]*len(test_df)
# test_df['lb4'] = [0]*len(test_df)
# test_df['ngames'] = [1]*len(test_df)




In [406]:
# ## dropping rookies 


# def drop_rookies(test_df):
#     new_df =pd.DataFrame(columns=())
#     for i in test_df.name.unique():
#         tmpdf = test_df.loc[test_df['name'] == i]
#         tmpdf = tmpdf.sort_values(['game_date'], ascending = [1]).reset_index(drop=True)
#         if tmpdf.loc[0,'pl_gc'] > 1:
#             new_df = pd.concat([new_df,tmpdf], axis=0).reset_index(drop=True)

#     return new_df

In [407]:
# test_df = drop_rookies(test_df)

In [408]:
# ### lb1,lb2,lb3,lb4,ngames,pl_gc
# def hist_data(tr_df,nm):
#     tdf = tr_df.loc[tr_df['name'] == nm]
#     tdf = tdf.sort_values(['game_date'], ascending = [0]).reset_index(drop= True)
#     rec = tdf.loc[0,['lb1','lb2','lb3','lb4','ngames','pl_gc']].tolist()
#     return rec

In [410]:
# def label_update(t1,i):
#     if t1.loc[i-1,'dfk_labels']  == 1:
#         t1.loc[i,'lb1'] = (t1.loc[i-1,'ngames']*t1.loc[i-1,'lb1'] + 1)/(t1.loc[i-1,'ngames'] + 1)
#         t1.loc[i,'lb2'] = (t1.loc[i-1,'ngames']*t1.loc[i-1,'lb2'] + 0)/(t1.loc[i-1,'ngames'] + 1)
#         t1.loc[i,'lb3'] = (t1.loc[i-1,'ngames']*t1.loc[i-1,'lb3'] + 0)/(t1.loc[i-1,'ngames'] + 1)
#         t1.loc[i,'lb4'] = (t1.loc[i-1,'ngames']*t1.loc[i-1,'lb4'] + 0)/(t1.loc[i-1,'ngames'] + 1)
#     elif t1.loc[i-1,'dfk_labels']  == 2:
#         t1.loc[i,'lb2'] = (t1.loc[i-1,'ngames']*t1.loc[i-1,'lb2'] + 1)/(t1.loc[i-1,'ngames'] + 1)
#         t1.loc[i,'lb1'] = (t1.loc[i-1,'ngames']*t1.loc[i-1,'lb1'] + 0)/(t1.loc[i-1,'ngames'] + 1)
#         t1.loc[i,'lb3'] = (t1.loc[i-1,'ngames']*t1.loc[i-1,'lb3'] + 0)/(t1.loc[i-1,'ngames'] + 1)
#         t1.loc[i,'lb4'] = (t1.loc[i-1,'ngames']*t1.loc[i-1,'lb4'] + 0)/(t1.loc[i-1,'ngames'] + 1)
#     elif t1.loc[i-1,'dfk_labels']  == 3:
#         t1.loc[i,'lb3'] = (t1.loc[i-1,'ngames']*t1.loc[i-1,'lb3'] + 1)/(t1.loc[i-1,'ngames'] + 1)
#         t1.loc[i,'lb1'] = (t1.loc[i-1,'ngames']*t1.loc[i-1,'lb1'] + 0)/(t1.loc[i-1,'ngames'] + 1)
#         t1.loc[i,'lb2'] = (t1.loc[i-1,'ngames']*t1.loc[i-1,'lb2'] + 0)/(t1.loc[i-1,'ngames'] + 1)
#         t1.loc[i,'lb4'] = (t1.loc[i-1,'ngames']*t1.loc[i-1,'lb4'] + 0)/(t1.loc[i-1,'ngames'] + 1)
#     elif t1.loc[i-1,'dfk_labels']  == 4:
#         t1.loc[i,'lb4'] = (t1.loc[i-1,'ngames']*t1.loc[i-1,'lb4'] + 1)/(t1.loc[i-1,'ngames'] + 1)
#         t1.loc[i,'lb1'] = (t1.loc[i-1,'ngames']*t1.loc[i-1,'lb1'] + 0)/(t1.loc[i-1,'ngames'] + 1)
#         t1.loc[i,'lb2'] = (t1.loc[i-1,'ngames']*t1.loc[i-1,'lb2'] + 0)/(t1.loc[i-1,'ngames'] + 1)
#         t1.loc[i,'lb3'] = (t1.loc[i-1,'ngames']*t1.loc[i-1,'lb3'] + 0)/(t1.loc[i-1,'ngames'] + 1)
#     return t1

In [411]:
# ### Updating the lb1 to ngame labels 

# def test_labels_update(test_df, tr_df):

#     new_df = pd.DataFrame(columns=())
#     for i in test_df.name.unique():
#         tmpdf = test_df.loc[test_df['name'] == i]
#         t1 = tmpdf.sort_values(['game_date'], ascending = [1]).reset_index(drop=True)
#         rec = hist_data(tr_df,i)
#         t1['lb1'] = [rec[0]]*len(t1)
#         t1['lb2'] = [rec[1]]*len(t1)
#         t1['lb3'] = [rec[2]]*len(t1)
#         t1['lb4'] = [rec[3]]*len(t1)
#         for j in range(len(t1)):
#             if j == 0:
#                 t1.loc[0,['lb1','lb2','lb3','lb4','ngames']] = rec[:-1]
#             else:
#                 ## label update 
#                 t1 = label_update(t1,j)
#                 t1.loc[j,'ngames'] = t1.loc[j-1,'ngames'] + t1.loc[j,'ngames']

#         new_df = pd.concat([new_df,t1]).reset_index(drop=True)
#     return new_df

In [412]:
# ### Updating the lb1 to ngame labels 

# def test_labels_update(test_df, tr_df):

#     new_df = pd.DataFrame(columns=())
#     for i in test_df.name.unique():
#         tmpdf = test_df.loc[test_df['name'] == i]
#         t1 = tmpdf.sort_values(['game_date'], ascending = [1]).reset_index(drop=True)
#         rec = hist_data(tr_df,i)
#         t1['lb1'] = [rec[0]]*len(t1)
#         t1['lb2'] = [rec[1]]*len(t1)
#         t1['lb3'] = [rec[2]]*len(t1)
#         t1['lb4'] = [rec[3]]*len(t1)
#         t1['ngames'] = [rec[4]]*len(t1)
# #         for j in range(len(t1)):
# #             if j == 0:
# #                 t1.loc[0,['lb1','lb2','lb3','lb4','ngames']] = rec[:-1]
# #             else:
# #                 ## label update 
# #                 t1 = label_update(t1,j)
# #                 t1.loc[j,'ngames'] = t1.loc[j-1,'ngames'] + t1.loc[j,'ngames']

#         new_df = pd.concat([new_df,t1]).reset_index(drop=True)
#     return new_df

In [413]:
# test_df = test_labels_update(test_df, tr_df)

In [15]:
# #### prob features team 


# def prob_features(tr_df):
#     new_df = pd.DataFrame(columns=())
#     for i in tr_df.name.unique():
#         tmpdf = tr_df.loc[tr_df['name'] == i]
#         ##tmpdf = tmpdf.sort_values(['game_date'], ascending=[1]).reset_index(drop=True)
#         t1 = tmpdf.sort_values(['game_date'],ascending =[1]).reset_index(drop=True)
#         for i in range(len(t1)):
#             if i == 0:
#                 t1.loc[i,['lb1','lb2','lb3','lb4','ngames']] = t1.loc[i,['lb1','lb2','lb3','lb4','ngames']] + [1,1,1,1,4]
#             else:
#                 t1.loc[i,['lb1','lb2','lb3','lb4','ngames']] = t1.loc[i,['lb1','lb2','lb3','lb4','ngames']] + t1.loc[i-1,['lb1','lb2','lb3','lb4','ngames']] 
#                 ## this means that the player is a rookie  
#         new_df = pd.concat([new_df,t1],axis=0).reset_index(drop=True)
    
#     return new_df

,off_pct,game_date,opp,starter_pos,offense,draftkings_points,team,game_num,game_location,name,...,result,pf,pa,ptotal,dfk_labels,lb1,lb2,lb3,lb4,ngames
0,100.0,2012-09-09,WAS,QB,72.0,26.6,NOR,1,home,Drew Brees,...,L,32,40,72,1,0.400000,0.200000,0.200000,0.200000,5
1,100.0,2012-09-16,CAR,QB,78.0,24.0,NOR,2,away,Drew Brees,...,L,27,35,62,1,0.500000,0.166667,0.166667,0.166667,6
2,100.0,2012-09-23,KAN,QB,60.0,20.5,NOR,3,home,Drew Brees,...,L,24,27,51,2,0.428571,0.285714,0.142857,0.142857,7
3,100.0,2012-09-30,GNB,QB,78.0,32.8,NOR,4,away,Drew Brees,...,L,27,28,55,1,0.500000,0.250000,0.125000,0.125000,8
4,100.0,2012-10-07,LAC,QB,71.0,32.6,NOR,5,home,Drew Brees,...,W,31,24,55,1,0.555556,0.222222,0.111111,0.111111,9
5,100.0,2012-10-21,TAM,QB,64.0,33.2,NOR,6,away,Drew Brees,...,W,35,28,63,1,0.600000,0.200000,0.100000,0.100000,10
6,100.0,2012-10-28,DEN,QB,61.0,15.7,NOR,7,away,Drew Brees,...,L,14,34,48,3,0.545455,0.181818,0.181818,0.090909,11
7,100.0,2012-11-05,PHI,QB,56.0,16.5,NOR,8,home,Drew Brees,...,W,28,13,41,3,0.500000,0.166667,0.250000,0.083333,12
8,100.0,2012-11-11,ATL,QB,62.0,22.8,NOR,9,home,Drew Brees,...,W,31,27,58,2,0.461538,0.230769,0.230769,0.076923,13
9,86.0,2012-11-18,OAK,QB,49.0,20.8,NOR,10,away,Drew Brees,...,W,38,17,55,2,0.428571,0.285714,0.214286,0.071429,14


In [12]:
# ### Feature Generation for the data set 
# ## 1st feature -- Average fantasy score for the position 

# def avg_fscore(tr_df):
#     year_list = np.arange(2012,2019,1) 

#     new_df = pd.DataFrame(columns=())
#     ##year_list = [2018]
#     for yr in year_list:
#         test = tr_df.loc[tr_df['year'] == yr]
#         hist_avg = tr_df['draftkings_points'].loc[tr_df['year'] != yr].mean()
#         test['avg_fscore'] = [hist_avg]*len(test) 
#         new_df = pd.concat([new_df,test], axis=0).reset_index(drop=True)
#     return new_df

In [13]:
# tr_df = avg_fscore(tr_df)

/Users/ag729/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [14]:
### Feature Generation for the data set 
## 1st feature -- Average fantasy score for the position 



def new_avg_fscore(tr_df):
    year_list = np.arange(2012,2019,1) 

    new_df = pd.DataFrame(columns=())
    ##year_list = [2018]
    for yr in year_list:
        tmpdf = tr_df.loc[tr_df['year'] == yr]
        hist_rec= tr_df['draftkings_points'].loc[tr_df['year'] != yr]
        hist_pts = hist_rec.sum()
        hist_len = len(hist_rec)
        for i in tmpdf.game_num.unique():
            test = tmpdf.loc[tmpdf['game_num'] == i]
            if i ==1:
                new_score = hist_pts/hist_len
            else:
                new_rec = tmpdf['draftkings_points'].loc[tmpdf['game_num'] < i]
                new_pts = new_rec.sum()
                new_len = len(new_rec)
                new_score = (hist_pts + new_pts)/(hist_len+new_len)

            test['new_avg_fscore'] = [new_score]*len(test) 
            new_df = pd.concat([new_df,test], axis=0).reset_index(drop=True)
    return new_df

In [15]:
tr_df = new_avg_fscore(tr_df)

/Users/ag729/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [16]:
#### Weekly average fantasy score for the position 

def week_avg(tr_df):

    year_list = np.arange(2012,2019,1) 

    #year_list = [2017]    

    new_df = pd.DataFrame(columns=())

    for yr in year_list:
        tmpdf = tr_df.loc[tr_df['year'] == yr]
        for i in tr_df.game_num.unique():
            test = tmpdf.loc[tmpdf['game_num'] == i]
            hist = tr_df['draftkings_points'].loc[(tr_df['year'] != yr) & (tr_df['game_num'] == i)].mean()
            test['week_avg'] = [hist]*len(test)
            new_df = pd.concat([new_df,test], axis=0).reset_index(drop=True)

    return new_df

In [17]:
tr_df = week_avg(tr_df)

/Users/ag729/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [18]:
#### Avg fantasy score for the position that year 

def year_avg_fscore(tr_df):

    year_list = np.arange(2012,2019,1) 

    

    new_df = pd.DataFrame(columns=())

    for yr in year_list:
        test = tr_df.loc[tr_df['year'] == yr]
        hist = tr_df['draftkings_points'].loc[(tr_df['year'] != yr)].mean()
        av_list = []
        new_list = [] 
        new_l1 = []
        for i in tr_df.game_num.unique():
            if i == 1:
                new_v = hist
            elif i > 1 and i <= 4 :
                val = test['draftkings_points'].loc[test['game_num'] < i].mean()
                ##new_v = 0.65*hist + 0.35*val ## these weights will be important in 
                new_v = val
            else:
#                 val = test['draftkings_points'].loc[test['game_num'] < i].mean()
#                 new_v = 0.35*hist + 0.65*val ## these weights will be important in 
                val = test['draftkings_points'].loc[test['game_num'] < i].mean()
                ##new_v = 0.10*hist + 0.90*val ## these weights will be important in 
                new_v = val
                
                
            tdf = test.loc[test['game_num'] == i]
            tdf['year_avg_fscore'] = [new_v]*len(tdf)
            ## new_list.append(new_v)
            new_df = pd.concat([new_df,tdf],axis=0).reset_index(drop=True)

        ##test['year_avg_fscore'] = new_list

        ## new_df = pd.concat([new_df,test],axis=0).reset_index(drop=True)
    return new_df

In [19]:
### add the year_avg_of_the_position feature 

tr_df = year_avg_fscore(tr_df)


/Users/ag729/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [20]:
## Player's fanatsy score in the last game 

def last_game_score(tr_df, yr):

    new_df = pd.DataFrame(columns=())

    ## getting the list of names
    tmpdf = tr_df.loc[tr_df['year'] == yr]
    for i in tmpdf.name.unique():
        last_game_score = []

#         if yr == 2012:
#             ## if we are in 2012.. last year avg or the first week value.. is the avg of that year 
#             ### lets assume average across the years for the position.. thus everyone is starting with the 
#             ### same baseline 
#             ## chk = tr_df.loc[(tr_df['year'] == yr) &(tr_df['name'] == i)]
#             lst_yr_avg = tr_df['draftkings_points'].mean()
#         else:
            ### the player's career average before this year.. if the player is a rookie then..
            ### avg for the position across the years 
            
        chk = tr_df.loc[(tr_df['year'] != yr) &(tr_df['name'] == i)]
        if len(chk) != 0:
            lst_yr_avg = chk['draftkings_points'].mean()
        else:
            ## if the player didn't play before then whatever was the average for the position 
            ##chk = tr_df.loc[(tr_df['year'] == yr-1)]
            lst_yr_avg = tr_df['draftkings_points'][tr_df['year'] != yr].mean()

        ## this years record for the player 
        tdf = tr_df.loc[(tr_df['year'] == yr) &(tr_df['name'] == i)]
        ### creating a list with last year's avg and this years scores from 1st game to n-1 game
        last_game_score = [lst_yr_avg] + tdf['draftkings_points'].tolist()[:-1]
        prev_yr_avg = [lst_yr_avg]*len(tdf)

        tdf['last_game_score'] = last_game_score
        ## tdf['prev_yr_avg'] = prev_yr_avg
        new_df = pd.concat([new_df,tdf], axis=0).reset_index(drop=True)
        ## the new df is the new training_df for that year 
    return new_df

In [21]:
yr_list = np.arange(2012,2019,1)

#### Training data adding features
train_data = pd.DataFrame(columns=())
for yr in yr_list:
    tmp = last_game_score(tr_df, yr)
    train_data = pd.concat([train_data,tmp], axis=0).reset_index(drop=True)


/Users/ag729/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [147]:
train_data

,game_date,opp,starter_pos,draftkings_points,team,game_num,game_location,name,year,dfk_labels,pl_gc,wrong_avg_fscore,avg_fscore,new_avg_fscore,week_avg,year_avg_fscore,last_game_score
0,2012-09-09,WAS,QB,26.6,NOR,1,home,Drew Brees,2012.0,1,1,18.766167,18.850395,18.850395,18.889655,18.850395,22.222472
1,2012-09-16,CAR,QB,24.0,NOR,2,away,Drew Brees,2012.0,1,2,18.767142,18.850395,18.835345,19.013158,17.566667,26.600000
2,2012-09-23,KAN,QB,20.5,NOR,3,home,Drew Brees,2012.0,2,3,18.768455,18.850395,18.831961,18.677027,18.069091,24.000000
3,2012-09-30,GNB,QB,32.8,NOR,4,away,Drew Brees,2012.0,1,4,18.763841,18.850395,18.823824,19.940397,18.095181,20.500000
4,2012-10-07,LAC,QB,32.6,NOR,5,home,Drew Brees,2012.0,1,5,18.763916,18.850395,18.848428,18.925503,18.807339,32.800000
5,2012-10-21,TAM,QB,33.2,NOR,6,away,Drew Brees,2012.0,1,6,18.763691,18.850395,18.834218,19.123270,18.561481,32.600000
6,2012-10-28,DEN,QB,15.7,NOR,7,away,Drew Brees,2012.0,3,7,18.770255,18.850395,18.853076,17.972185,18.890741,33.200000
7,2012-11-05,PHI,QB,16.5,NOR,8,home,Drew Brees,2012.0,3,8,18.769955,18.850395,18.840950,20.560510,18.726596,15.700000
8,2012-11-11,ATL,QB,22.8,NOR,9,home,Drew Brees,2012.0,2,9,18.767592,18.850395,18.822160,19.362658,18.523256,16.500000
9,2012-11-18,OAK,QB,20.8,NOR,10,away,Drew Brees,2012.0,2,10,18.768342,18.850395,18.813042,18.015789,18.457322,22.800000


In [22]:
## Player Career Average 

def player_career_avg(train_data):

    year_list = np.arange(2012,2019,1)

    fin_df = pd.DataFrame(columns=())

    for yr in year_list:
        new_df = pd.DataFrame(columns=())

        ## getting the list of names
        tmpdf = train_data.loc[train_data['year'] == yr]
        for i in tmpdf.name.unique():
            last_game_score = []


            ### the player's career average before this year.. if the player is a rookie then..
            ### avg for the position across the years 

            chk = train_data.loc[(train_data['year'] != yr) &(train_data['name'] == i)]
            if len(chk) != 0:
                hist = chk['draftkings_points'].mean()
            else:
                hist = train_data['draftkings_points'].mean()
                hist = train_data['draftkings_points'].loc[train_data['year'] != yr].mean()
            ## test = tr_df.loc[tr_df['year'] == yr]

            test = tmpdf.loc[tmpdf['name'] == i]
            new_list = []
            game_list = test.game_num.unique().tolist()
            for j in range(len(game_list)):
                if j+1 == 1:
                    new_v = hist
                elif j+1 > 1 and j+1 <= 4 :
                    val = test['draftkings_points'].loc[test['game_num'] < game_list[j]].mean()
                    new_v = 0.50*hist + 0.50*val ## these weights will be important in 
                else:
                    val = test['draftkings_points'].loc[test['game_num'] < game_list[j]].mean()
                    #new_v = 0.20*hist + 0.80*val ## these weights will be important in 
                    new_v = 0.50*hist + 0.50*val ## these weights will be important in 
                    
                ###### Keeping the weights a little more steep here because these are individual averages
                ## and the average for the year should have more weight.. then career average


                new_list.append(new_v)
            tdf = test.sort_values(['game_num'], ascending=[1])
            tdf['player_career_avg'] = new_list
            ## new_list.append(new_v)
            new_df = pd.concat([new_df,tdf],axis=0).reset_index(drop=True)


        fin_df = pd.concat([fin_df,new_df], axis=0).reset_index(drop=True)


    return fin_df



In [23]:
train_data = player_career_avg(train_data)

In [24]:
### Position average fantasy score against the opponent 
def pos_avg_ag_op(train_data):

    year_list = np.arange(2012,2019,1) 

    

    fin_df = pd.DataFrame(columns=())

    for yr in year_list:
        new_df = pd.DataFrame(columns=())
        tmpdf = train_data.loc[train_data['year'] == yr]
        ##hist_avg = tr_df['draftkings_points'].loc[tr_df['year'] != yr].mean()
        av_list = []
        new_list = [] 
        new_l1 = []
        for i in tr_df.opp.unique():
            hist_avg = train_data['draftkings_points'].loc[(train_data['year'] != yr) & (train_data['opp'] == i)].mean()
            test = tmpdf.loc[tmpdf['opp'] == i]
            test['pos_avg_ag_op'] = [hist_avg]*len(test)
            new_df = pd.concat([new_df,test],axis=0).reset_index(drop=True)
        
            
#             game_list = test.game_num.unique().tolist()
#             for j in range(len(game_list)):
            
#                   if j+1 == 1:
#                     new_v = hist
#                 elif j+1 > 1 and j+1 <= 4 :
#                     val = test['draftkings_points'].loc[test['game_num'] < game_list[j]].mean()
#                     new_v = 0.50*hist + 0.50*val ## these weights will be important in 
#                 else:
#                     val = test['draftkings_points'].loc[test['game_num'] < game_list[j]].mean()
#                     new_v = 0.20*hist + 0.80*val ## these weights will be important in 

#                 ###### Keeping the weights a little more steep here because these are individual averages
#                 ## and the average for the year should have more weight.. then career average


#                 new_list.append(new_v)
#             tdf = test.sort_values(['game_num'], ascending=[1])
#             tdf['player_career_avg'] = new_list
#             ## new_list.append(new_v)


        fin_df = pd.concat([fin_df,new_df], axis=0).reset_index(drop=True)


        ##test['year_avg_fscore'] = new_list

        ## new_df = pd.concat([new_df,test],axis=0).reset_index(drop=True)
    return fin_df

In [25]:
train_data = pos_avg_ag_op(train_data)

/Users/ag729/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [27]:
### Player average against the opponent

def ply_avg_ag_opp(train_data):
    
    year_list = np.arange(2012,2019,1)

    fin_df = pd.DataFrame(columns=())
    ## year_list = [2018]
    for yr in year_list:
        new_df = pd.DataFrame(columns=())

        ## getting the list of names
        tmpdf = train_data.loc[train_data['year'] == yr]
        for i in tmpdf.name.unique():


            ### the player's career average before this year.. if the player is a rookie then..
            ### avg for the position across the years 

            test = tmpdf.loc[tmpdf['name'] == i]

            opp_list = test.opp.tolist()

            opp_avg_list = []
            for j in range(len(opp_list)):
                ## player games besides this year against this particular opponent 
                chk = train_data.loc[(train_data['opp'] == opp_list[j]) & (train_data['year'] != yr) & (train_data['name'] == i) ]
                if len(chk) != 0:
                    hist = chk['draftkings_points'].mean()
                else:
                    ## if the player hasn't played against this team earlier then the player's career average 
                    ## updated for that season.. that has been calcualted
                    ## earlier will be used 
                    ## NOT SURE IF THIS MAKES A WHOLE LOT OF SENSE... AS IT IS DEPNDENT ON ANOTHER FEATURE 
                    ### FOR THE ROOKIES..WHY NOT THE AVG SCORE AGAINST THAT OPPONENT?
                    ### I THINK EITHER COULD BE USED...
                    ## chk = test.loc[(test['opp'] == i) ]
                    hist = test['player_career_avg'].mean()

                opp_avg_list.append(hist)

            test['ply_avg_ag_opp'] = opp_avg_list
            new_df = pd.concat([new_df,test], axis=0).reset_index(drop=True)

        fin_df = pd.concat([fin_df,new_df],axis = 0).reset_index(drop=True)

    return fin_df 

In [28]:
 
train_data = ply_avg_ag_opp(train_data)

/Users/ag729/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [29]:
##### PLayers home vs away averages

def home_away_split(train_data):

    year_list = np.arange(2012,2019,1)

    fin_df = pd.DataFrame(columns=())
    ##year_list = [2018]
    for yr in year_list:
        new_df = pd.DataFrame(columns=())

        ## getting the list of names
        tmpdf = train_data.loc[train_data['year'] == yr]
        for i in tmpdf.name.unique():


            ### the player's career average before this year.. if the player is a rookie then..
            ### avg for the position across the years 

            test = tmpdf.loc[tmpdf['name'] == i]

            new_list = []
            game_list = test.game_num.unique().tolist()

            for j in range(len(game_list)):
                ## player games besides this year against this particular opponent 
                chk = test['game_location'][test['game_num'] == game_list[j]].values[0]
                if chk == 'home':
                    fildf = train_data.loc[(train_data['name'] == i) & (train_data['game_location'] == 'home') & (train_data['year'] != yr)]
                    if len(fildf) != 0:
                        hist = fildf['draftkings_points'].mean()
                    else:
                        fildf = train_data.loc[(train_data['game_location'] == 'home') & (train_data['year'] != yr)]
                        hist = fildf['draftkings_points'].mean()
                if chk == 'away':
                    fildf = train_data.loc[(train_data['name'] == i) & (train_data['game_location'] == 'away') & (train_data['year'] != yr)]
                    if len(fildf) != 0:
                        hist = fildf['draftkings_points'].mean()
                    else:
                        fildf = train_data.loc[(train_data['game_location'] == 'away') & (train_data['year'] != yr)]
                        hist = fildf['draftkings_points'].mean()

                new_list.append(hist)

            test['home_away_split'] = new_list
            new_df = pd.concat([new_df,test], axis=0).reset_index(drop=True)

        fin_df = pd.concat([fin_df,new_df],axis = 0).reset_index(drop=True)

    return fin_df


In [30]:
train_data = home_away_split(train_data)

/Users/ag729/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [31]:
########### Classifier data

qb_dat = train_data.copy()

### Labeling Names and converting them to index 
from sklearn import preprocessing 
le = preprocessing.LabelEncoder()
le.fit(qb_dat['name'].tolist())
list(le.classes_)
qid = le.transform(qb_dat['name'].tolist()) 
qb_dat['qid'] = qid

In [35]:
## Removing data that isn't needed
qb_dat = qb_dat.drop(['game_date', 'starter_pos','game_num','opp','team','game_location',
                     'name','draftkings_points','avg_fscore','qid'], axis=1)




In [37]:
### Dividing into Test and Training data
Xs = qb_dat.loc[qb_dat['year'] != 2018]
Ys = qb_dat.loc[qb_dat['year'] == 2018]

Xs = Xs.drop(['year'], axis =1)
Ys = Ys.drop(['year'], axis = 1)

X_train = Xs.ix[:, ~Xs.columns.isin(['dfk_labels'])]
X_test = Ys.ix[:, ~Ys.columns.isin(['dfk_labels'])]

y_train = Xs['dfk_labels']
y_test = Ys['dfk_labels']

/Users/ag729/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.
/Users/ag729/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [40]:
#### Naive Bayes Prediction and Testing on Test Data
from sklearn.naive_bayes import GaussianNB 
gnb = GaussianNB().fit(X_train, y_train) 
gnb_predictions = gnb.predict(X_test)

from sklearn.metrics import accuracy_score
accuracy_score(y_test,gnb_predictions)

### It's about 6 percent better than random guess, which was expected 

0.3151041666666667